In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D,MaxPooling2D
import matplotlib.pyplot as plt


In [ ]:
# download the dataset , you can also check in the readme

In [ ]:
import zipfile

# Path to your full dataset ZIP
zip_path = 'chest-xray-pneumonia.zip'

# Directory to extract into
extract_dir = 'chest_xray_full'

# Unzip
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Dataset unzipped to:", extract_dir)


In [ ]:
import matplotlib.image as mpimg

img_path = "/content/chest_xray_full/chest_xray/chest_xray/train/PNEUMONIA/person1000_bacteria_2931.jpeg"

image = mpimg.imread(img_path)
plt.imshow(image,cmap = 'gray')
plt.axis('off')
plt.show()



In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

base_dir ='chest_xray_full/chest_xray'

traindata_image = ImageDataGenerator(
    rescale =1./255,
    shear_range =0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    validation_split = 0.2
)

# train generator

train_generator = traindata_image.flow_from_directory(
    f'{base_dir}/train',
    target_size = (150 ,150),
    batch_size = 32,
    class_mode = 'binary',
    subset ='training'
)

# Validation generator so they can test also to the validation same type
validation_generator = traindata_image.flow_from_directory(
    f'{base_dir}/train',
    target_size=(150,150),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

In [ ]:
model = Sequential([
    Conv2D(32,(3*3),activation ='relu', input_shape =(150,150,3)),
    MaxPooling2D(2,2),

    Conv2D(64,(3*3),activation ='relu'),
    MaxPooling2D(2,2),

    Conv2D(128,(3*3),activation ='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')


])

In [9]:
# Compiling the model

model.compile(
    optimizer ='adam',
    loss ='binary_crossentropy',
    metrics = ['accuracy']
)

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples,
    epochs=5
)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    f'{base_dir}/test',
    target_size=(150,150),
    batch_size=1,        # 1 image at a time for easy display
    class_mode='binary',
    shuffle=False         # keep order same for checking
)

In [ ]:
import numpy as np

# Predict classes for test images
preds = model.predict(test_generator)
pred_classes = (preds > 0.5).astype(int)  # 0 = NORMAL, 1 = PNEUMONIA


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Display first 5 test images with predicted labels
for i in range(5):
    img_path = test_generator.filepaths[i]
    img = mpimg.imread(img_path)

    plt.imshow(img, cmap='gray')
    plt.title(f'Predicted: {"PNEUMONIA" if pred_classes[i]==1 else "NORMAL"}')
    plt.axis('off')
    plt.show()


In [ ]:
true_classes = test_generator.classes

for i in range(5):
    img_path = test_generator.filepaths[i]
    img = mpimg.imread(img_path)

    plt.imshow(img, cmap='gray')
    plt.title(f'True: {"PNEUMONIA" if true_classes[i]==1 else "NORMAL"} | Pred: {"PNEUMONIA" if pred_classes[i]==1 else "NORMAL"}')
    plt.axis('off')
    plt.show()
